<a href="https://colab.research.google.com/github/Linoh12/LearningLSTM/blob/main/LSTM_Lincoln.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Import

In [ ]:
!pip install scikit-learn-intelex

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.7/128.7 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.0/14.0 MB 46.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.3/75.3 MB 13.5 MB/s eta 0:00:00


In [ ]:
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torchvision

import sklearn
from sklearn.model_selection import train_test_split as tts
from sklearn.metrics import accuracy_score
from sklearnex import patch_sklearn
patch_sklearn()

import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

from pathlib import Path

torch.__version__

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


'2.0.1+cu118'

In [ ]:
fp = 'raw_data.csv'
data = pd.read_csv(fp)
data

FileNotFoundError: ignored

In [ ]:
timestamps = np.unique(data['Timestamp'])
timestamps

In [ ]:
train_timestamps = timestamps[:5]
test_timestamps = timestamps[1:6]
train_timestamps, test_timestamps

In [ ]:
timestamp_train_data = data[np.isin(data['Timestamp'], train_timestamps)]
timestamp_train_data

In [ ]:
timestamp_train_data['present'] = 1
arr = []
for row in timestamp_train_data.to_numpy():
  temp = row
  temp[0],temp[1],temp[-1] = temp[1],temp[0],0
  arr.append(temp)

timestamp_train_data = pd.concat((timestamp_train_data, pd.DataFrame(arr, columns=timestamp_train_data.columns)), axis=0)
timestamp_train_data

In [ ]:
arr = []
for time in train_timestamps:
  arr.append(timestamp_train_data[timestamp_train_data['Timestamp'] == time].values)

train_data_seq = torch.Tensor(arr)
train_data_seq

In [ ]:
train_data_seq = torch.transpose(train_data_seq, 0, 1).numpy()
train_data_seq.shape

In [ ]:
timestamp_test_data = data[np.isin(data['Timestamp'], test_timestamps)]

timestamp_test_data['present'] = 1
arr = []
for row in timestamp_test_data.to_numpy():
  temp = row
  temp[0],temp[1],temp[-1] = temp[1],temp[0],0
  arr.append(temp)

timestamp_test_data = pd.concat((timestamp_test_data, pd.DataFrame(arr, columns=timestamp_test_data.columns)), axis=0).reset_index(drop=True)
timestamp_test_data

In [ ]:
arr = []
for time in test_timestamps:
  arr.append(timestamp_test_data[timestamp_test_data['Timestamp'] == time].values)

test_data_seq = torch.Tensor(arr)
test_data_seq

In [ ]:
test_data_seq = torch.transpose(test_data_seq, 0, 1).numpy()
test_data_seq.shape

# Model Building

In [ ]:
#Device Agnostic Code
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

In [ ]:
#Hyper Parameters
input_size = 10
num_layers = 5
sequence_length = 5

hidden_size = 10
#num_classes = 1

num_epochs = 2000 #3000
batch_size = 32
learning_rate = 0.001 #0.00025

## Data Prep

In [ ]:
class dataset(torch.utils.data.Dataset):
  def __init__(self,x,y):
    self.x = torch.tensor(x,dtype=torch.float32).to(device)
    self.y = torch.tensor(y,dtype=torch.float32).to(device)
    self.length = self.x.shape[0]

  def __getitem__(self,idx):
    return self.x[idx],self.y[idx]
  def __len__(self):
    return self.length

target = ['present']

X_train, y_train = train_data_seq[:,:,:-1], train_data_seq[:,:,-1] #np.expand_dims(train_data_seq[:,:,-1], axis=2)
X_test, y_test = test_data_seq[:,:,:-1], test_data_seq[:,:,-1] #np.expand_dims(test_data_seq[:,:,-1], axis=2)

trainset = dataset(X_train,y_train)
testset = dataset(X_test,y_test)

In [ ]:
train_loader = torch.utils.data.DataLoader(trainset, batch_size=batch_size)
test_loader = torch.utils.data.DataLoader(testset, batch_size=batch_size)

In [ ]:
asdf = iter(train_loader)
x,y = next(asdf)
x.shape,y.shape

In [ ]:
asdf = iter(test_loader)
x,y = next(asdf)
x.shape,y.shape

## Model

In [ ]:
torch.manual_seed(42)

In [ ]:
class RNN(nn.Module):
  def __init__(self, input_size, hidden_size, num_layers):
    super(RNN, self).__init__()
    self.input_size = input_size
    self.hidden_size = hidden_size
    self.num_layers = num_layers
    # self.num_classes = num_classes
    self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
    self.fc = nn.Linear(hidden_size,1)

    self.s1 = nn.Sigmoid()


  def forward(self, x):
      # h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
      # c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)

      out,_ = self.lstm(x) # x -> (batch_size, seq, input_size)
      # out: (batch_size, seq_length, hidden_size) # from rnn
      # out(N, 5, 5)
      out = out[:, -1, :]
      out = self.fc(out)
      out = self.s1(out)
      return out


In [ ]:
model = RNN(input_size, hidden_size, num_layers).to(device)

#loss and optimizer
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

## Training

In [ ]:
#Training Loop
model.train()

n_total_steps = len(train_loader)
for epoch in range(num_epochs):
  for i, (x_train,y_train) in enumerate(train_loader):
    # print(f'Iter {i+1}, x_train:{x_train[:5]}, y_train:{y_train[:5]}')

    #forward
    # print('x_train before reshaping: ',x_train.shape)
    x_train = x_train.reshape(-1, sequence_length, input_size).to(device)
    # print('x_train after reshaping: ', x_train.shape)
    outputs = model(x_train)

    y_train = y_train[:,-1].unsqueeze(dim=1)

    # print(outputs,y_train,sep='\n')
    # print(outputs.shape, y_train.shape)
    loss = criterion(outputs, y_train)

    #backward
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    if (epoch+1)%100 == 0:
      print(f'Epoch {epoch+1}/{num_epochs}, step {i+1}/{n_total_steps}, loss = {loss.item():.5f}')

      if (i+1)==n_total_steps:
        print('')

## Testing

In [ ]:
model.input_size, model.num_layers, model.hidden_size

In [ ]:
def binary_threshold(x, threshold= 0.5):
  # print(x)
  threshold = torch.tensor([threshold]).to(device)
  results = (x>threshold).float()*1
  # print(results)
  return results

In [ ]:
#Testing Loop for Training Data
model.eval()
n_correct = 0
n_samples = 0
with torch.inference_mode():
  for i, (x_train,y_train) in enumerate(train_loader):
    x_train = x_train.reshape(-1, sequence_length, input_size).to(device)
    y_train = y_train[:,-1].unsqueeze(dim=1).to(device)
    outputs = model(x_train)
    outputs = binary_threshold(outputs)

    # print(outputs,y_train)
    # print(outputs.shape, y_train.shape)

    n_samples += y_train.shape[0] * y_train.shape[1]

    n_correct += (outputs == y_train).sum().item()
    # print((outputs==y_train).sum())
    # print(n_correct, n_samples)

acc = 100* n_correct/n_samples
print(f'Training Accuracy = {acc:.3f}%')

In [ ]:
#Testing Loop for Testing Data
model.eval()
n_correct = 0
n_samples = 0
with torch.inference_mode():
  for i, (x_test,y_test) in enumerate(test_loader):
    x_test = x_test.reshape(-1, sequence_length, input_size).to(device)
    y_test = y_test.to(device)
    outputs = model(x_test)
    outputs = binary_threshold(outputs)

    # print(outputs,y_test)
    # print(outputs.shape, y_test.shape)

    ### If we only use the new unseen data
    outputs = outputs[:,-1]
    y_test = y_test[:,-1]

    # print(outputs,y_test)
    # print(outputs.shape, y_test.shape)

    # n_samples += y_test.shape[0] * y_test.shape[1]
    n_samples += y_test.shape[0]

    n_correct += (outputs == y_test).sum().item()

    print(f'Cumulative: {n_correct}/{n_samples}')

acc = 100* n_correct/n_samples
print(f'Testing Accuracy = {acc:.3f}%')